In [ ]:
%pip install -q matplotlib pandas tqdm m2stitch
%pip install -q git+https://github.com/yfukai/BaSiCpy@autotune

In [ ]:
from basicpy import BaSiC, datasets
from matplotlib import pyplot as plt
import numpy as np
from hyperactive import Hyperactive
import pandas as pd
from m2stitch import stitch_images

plt.rcParams["font.family"] = ""
plt.figure(figsize=(1, 1))
plt.plot(range(2))

In [ ]:
brain_wsi_image = datasets.wsi_brain()

In [ ]:
brain_wsi_image.shape

In [ ]:
def compose_image(img):
    img_composed = np.empty(
        (
            img.shape[1] * 7,
            img.shape[2] * 9,
        )
    )

    y = 0
    x = img.shape[2] * 8

    rows = []
    cols = []
    for j, im in enumerate(img):
        img_composed[y : y + img.shape[1], x : x + img.shape[2]] = im
        rows.append(y // img.shape[1])
        cols.append(x // img.shape[2])
        if (y // img.shape[2]) % 2 == 0:
            x -= img.shape[2]
            if x < 0:
                x = 0
                y += img.shape[1]
        else:
            x += img.shape[2]
            if x > img.shape[2] * 8:
                x = img.shape[2] * 8
                y += img.shape[1]
    return img_composed, rows, cols

In [ ]:
composed, rows, cols = compose_image(brain_wsi_image)
plt.imshow(composed, vmin=970, vmax=10000)
plt.colorbar()

In [ ]:
basic = BaSiC(
    get_darkfield=True,
)

In [ ]:
print(
    basic.smoothness_flatfield, basic.smoothness_darkfield, basic.sparse_cost_darkfield
)

In [ ]:
transformed1 = basic.fit_transform(brain_wsi_image)
composed, rows, cols = compose_image(transformed1)
plt.imshow(composed, vmin=970, vmax=10000)
plt.colorbar()

In [ ]:
basic.autotune(brain_wsi_image, early_stop=True, n_iter=100)

In [ ]:
print(
    basic.smoothness_flatfield, basic.smoothness_darkfield, basic.sparse_cost_darkfield
)

In [ ]:
transformed2 = basic.fit_transform(brain_wsi_image)
composed, rows, cols = compose_image(transformed2)
plt.imshow(composed, vmin=970, vmax=10000)
plt.colorbar()

In [ ]:
df, props = stitch_images(
    transformed1, rows, cols, row_col_transpose=False, ncc_threshold=0.2
)

In [ ]:
stitched_images = []

for image in [brain_wsi_image, transformed1, transformed2]:
    stitched_image = np.zeros(
        (
            image.shape[1] + df["y_pos"].max(),
            image.shape[2] + df["x_pos"].max(),
        )
    )
    for (_, row), im in zip(df.iterrows(), image):
        stitched_image[
            row["y_pos"] : row["y_pos"] + im.shape[0],
            row["x_pos"] : row["x_pos"] + im.shape[1],
        ] = im
    stitched_images.append(stitched_image)

In [ ]:
for name, stitched_image in zip(
    ["original", "default parameter", "tuned parameter"], stitched_images
):
    plt.figure(figsize=(5, 5))
    plt.title(name)
    plt.imshow(stitched_image, vmin=970, vmax=10000)
    plt.colorbar()